## Example of Landlab 'landslide' component

This tutorial provides an example driver code that runs Landlab's 'landslide' component.
What a driver does is gathers input data, run the Landlab landslide component, and visualizes data and outputs.
Landlab is a Python-based landscape modeling environment and the landslide component is one of many components available for users to access and link together to build their own landscape model.
For more information about Landlab, see http://landlab.github.io/#/.

Input data is provide by the user and consists of elevation from a DEM to provide topographic traits such as slope, contributing area, and flow direction. The user also supplies soil characteristics derived from a soil survey,
land cover, or other sources, including transmissivity, cohesion, internal angle of friction, density, and thickness. 
Data for this example can be acquired from https://www.HydroShare.org under the resource "Thunder Creek Landlab Landslide Example".

Method calculates factor-of-safety stability index by using node specific parameters, creating distributions of these parameters, and calculating the index by sampling these distributions 'n' times. The index is calculated from the 'infinite slope stabilty factor-of-safety equation' in the format of Pack RT, Tarboton DG, and Goodwin CN (1998) 'The SINMAP approach to terrain stability mapping.'

Output includes figures of relative wetness, mean factor-of-safety, and probability of failure based on 
factor-of-safety calculations within a Monte Carlo simulation.

This version allows users to provide just a minimum and maximum recharge that us used in a uniform distribution rather than a spatially distributed recharge field. Thus, it is for testing and teaching purposes. It is design to run from the
same directory where the data files are located.

@author: R.Strauch and E.Istanbulluoglu - Univerity of Washington Created on Thu Aug 20 16:47:11 2015 Last edit July 20, 2016


### To run example

To run this example, click in each shaded cell below and "shift + enter" to run each cell.
Alternatively, you can run groups of cells by clicking "Cell" on the menu above and selecting you run option.  This is also where you can clear outputs from previous runs.

If an error occurs, try "Restart" the kernel by clicking "Kernel" on the menu above.

### Import libraries and components

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pylab
from landlab.plot.imshow import imshow_node_grid
from landlab.io import read_esri_ascii
from landlab.components.landslides import LandslideProbability
from utilities import hydroshare
%matplotlib inline

NOTE - Warnings about matplotlib and gFlex are OKAY and won't affect running the component.

Next we need to establish a secure connection with HydroShare. This is done by simply instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets environment variables for several parameters that may useful to you:

1. Your username
2. The ID of the resource which launched the notebook
3. The type of resource that launched this notebook
4. The url for the notebook server.

In [ ]:
# establish a secure connection to HydroShare
hs = hydroshare.hydroshare()

### Retrieve a raster resource using its ID

This example uses temperature data that is stored in HydroShare at the following url: https://www.hydroshare.org/resource/0e49df4b97f94247a8d52bac4adeb14a/ . The data for our processing routines can be retrieved using the getResourceFromHydroShare function by passing in the global identifier from the url above.

In [ ]:
# get some resource content. The resource content is returned as a dictionary
content = hs.getResourceFromHydroShare('0e49df4b97f94247a8d52bac4adeb14a')

### Load grid, fields, and constants

Reading a esri_ascii file from ArcGIS sets up the RasterModelGrid and assign elevation field to nodes. Nodes are the center point of grid cells or pixels that are 30 m by 30 m in this example.
The ascii (txt) files are located in the same directory where this notebook is run.
This might take 30 seconds.

In [ ]:
(grid, z) = read_esri_ascii(hs.content['thun_elevation.txt'],
                          name='topographic__elevation')
grid.at_node.keys()

Check what other fields you need for this 'landslide' component.  Fields are data values that are assigned to each node, providing spatial variability to landscape.

In [ ]:
sorted(LandslideProbability.input_var_names)

Load and add other fields to your grid. This will take 1  to 2 minutes, depending on how large is your study domain. In this example, there are 1,543,616 nodes, which covers 1,389 sq km (536 sq mi) in this example.  To confirm, try the code below.

In [ ]:
grid.number_of_nodes

In [ ]:
(grid1, slope) = read_esri_ascii(hs.content['thun_slope.txt'])
grid.add_field('node', 'topographic__slope', slope)
(grid1, ca) = read_esri_ascii(hs.content['thun_sp_ca.txt'])
grid.add_field('node', 'topographic__specific_contributing_area', ca)
(grid1, T) = read_esri_ascii(hs.content['thun_trans.txt'])
grid.add_field('node', 'soil__transmissivity', T)
(grid1, C) = read_esri_ascii(hs.content['thun_cmode.txt'])
C[C == 0.0] = 1.0  # ensure not 0 Pa for use in distributions generation
grid.add_field('node', 'soil__mode_total_cohesion', C)
(grid1, C_min) = read_esri_ascii(hs.content['thun_cmin.txt'])
grid.add_field('node', 'soil__minimum_total_cohesion', C_min)
(grid1, C_max) = read_esri_ascii(hs.content['thun_cmax.txt'])
grid.add_field('node', 'soil__maximum_total_cohesion', C_max)
(grid1, phi) = read_esri_ascii(hs.content['thun_phi.txt'])
grid.add_field('node', 'soil__internal_friction_angle', phi)
(grid1, Hs) = read_esri_ascii(hs.content['thun_soil_dpt.txt'])
grid.add_field('node', 'soil__thickness', Hs)

Note - Values of -9999 are areas that have no data or are outside our area of interest, which are within the input txt files.

Create a constant field (same value at every node) for soil density.

In [ ]:
grid['node']['soil__density'] = 2000*np.ones(grid.number_of_nodes)

Check the units of the cohesion fields or any other input variable with the command below.

In [ ]:
LandslideProbability.var_units('soil__mode_total_cohesion')

Add the watershed boundary and mapped landslides to our grid for plot overlays.  This takes about 30 seconds.

In [ ]:
(grid1, watershed) = read_esri_ascii(hs.content['thun_wsline.txt'])
grid.add_field('node', 'Thunder_Creek_watershed', watershed)
(grid1, slides) = read_esri_ascii(hs.content['thun_ls_type.txt'])
grid.add_field('node', 'landslides', slides)

### Set boundary conditions

Boundary conditions are where we want to limit our analysis, such as the areas that did not have -9999 values.  We can also add an analysis to a subset of our domain by using a mask that has value = 1 for nodes we want to include and value = -9999 for the nodes we want to exclude in our analysis.

In [ ]:
(grid1,mask) = read_esri_ascii(hs.content['thun_area_msk.txt']) # to limit analysis
grid.add_field('node', 'Thunder_Area_mask', mask)

Set nodes to closed when field value is -9999 (no data) or when slope or soil thickness is zero,
which causes errors from division by zero when calculating factor-of-safety within the landslide component.

In [ ]:
grid.set_nodata_nodes_to_closed(grid.at_node['topographic__elevation'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node['topographic__slope'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node['topographic__slope'], 0.0)
grid.set_nodata_nodes_to_closed(grid.at_node[
        'topographic__specific_contributing_area'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node['soil__transmissivity'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node[
        'soil__mode_total_cohesion'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node[
        'soil__minimum_total_cohesion'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node[
        'soil__maximum_total_cohesion'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node[
        'soil__internal_friction_angle'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node['soil__thickness'], -9999)
grid.set_nodata_nodes_to_closed(grid.at_node['soil__thickness'], 0.0)
grid.set_nodata_nodes_to_closed(grid.at_node['Thunder_Area_mask'], -9999)

Select the minimum and maximum annual peak recharge for your location in mm/day.  This represents the range of the wettest conditions expected annually, which is the severest soil-saturated conditions likely to occur at least once a year.  High soil saturation may occur more frequently than one day a year, thus, the instability index based on recharge is likely conservative.

In [ ]:
groundwater__recharge_minimum = 14.3
groundwater__recharge_maximum = 44.3
groundwater__recharge_maximum

### Run landslide component to calculate probability of failure

The landslide component employes the infinite slope model to calculate factor-of-safety values using a Monte Carlo simulation approach, which randomly selects input values from parameter distributions.  You can pick the number of iterations to run Monte Carlo simulations.  The higher the number of iteration, the longer the program runs, but the more precise the probability of failure result becomes.

In [ ]:
number_of_simulations = 25

Instantiate the landslide component.  This is where you access the component and provide you user-specified input parameters.

In [ ]:
LS_prob = LandslideProbability(
    grid, number_of_simulations=number_of_simulations,
    groundwater__recharge_minimum=groundwater__recharge_minimum,
    groundwater__recharge_maximum=groundwater__recharge_maximum)

Check what output you should expect when you run the component.

In [ ]:
sorted(LS_prob.output_var_names)

Run component for all core nodes in domain (977,394 in this example) by passing data from fields we imported earlier and 
user-specified parameters above to the FactorofSafety class within the component.  Core nodes are the nodes (grid cells) that we want our analysis to work on.
Running component also populates storage arrays with calculated values from component to use for plotting.  This can take 5 minute or more depending on size of domain and number of simulations specified above (default is 250).

In [ ]:
LS_prob.calculate_landslide_probability()

Review outputs from the component, such as the calculated factor-of-safety
values of only ONE core node.  This data is available at all core nodes.

In [ ]:
core_nodes = LS_prob.grid.core_nodes
LS_prob.landslide__factor_of_safety_histogram[core_nodes[10]]
# 10 is an index of the 11th core node, not node id

### Make plots and figures

Set plotting parameters

In [ ]:
core_nodes
mpl.rcParams['xtick.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 15
mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 15

Plot elevation overlain with watershed boundary and countours

In [ ]:
plt.figure('Elevations from the DEM [m]')
watershed_line = grid.at_node['Thunder_Creek_watershed'] < 0.0
overlay_watershed = np.ma.array(grid.at_node['Thunder_Creek_watershed'],
                                mask=watershed_line)
imshow_node_grid(grid, 'topographic__elevation', cmap='terrain',
                 grid_units=('coordinates', 'coordinates'),
                 shrink=0.75, var_name='Elevation', var_units='m')
elev = grid.node_vector_to_raster(z)
cs = pylab.contour(elev, extent=[0,42500, 0,32500], hold='on', colors='black')
manual_locals = [(11000,11000), (14000,30000), (20000,8000), (36000,3000)]
pylab.clabel(cs, inline=True, fmt='%1i', fontsize=10, manual=manual_locals)
imshow_node_grid(grid, overlay_watershed, color_for_closed='None',
                 allow_colorbar=False, cmap='spring')

This map shows the watershed boundaries in pink over topography.

Plot slope overlain with mapped landslide types and contours. Takes about a minute.

In [ ]:
plt.figure('Landslides')
ls_mask1 = grid.at_node['landslides'] != 1.0
ls_mask2 = grid.at_node['landslides'] != 2.0
ls_mask3 = grid.at_node['landslides'] != 3.0
ls_mask4 = grid.at_node['landslides'] != 4.0
overlay_landslide1 = np.ma.array(grid.at_node['landslides'], mask=ls_mask1)
overlay_landslide2 = np.ma.array(grid.at_node['landslides'], mask=ls_mask2)
overlay_landslide3 = np.ma.array(grid.at_node['landslides'], mask=ls_mask3)
overlay_landslide4 = np.ma.array(grid.at_node['landslides'], mask=ls_mask4)
imshow_node_grid(grid, 'topographic__slope', cmap='pink',
                 grid_units=('coordinates', 'coordinates'), vmax=2.,
                 shrink=0.75, var_name='Slope', var_units='m/m')
imshow_node_grid(grid, overlay_landslide1, color_for_closed='None',
                 allow_colorbar=False, cmap='cool')
imshow_node_grid(grid, overlay_landslide2, color_for_closed='None',
                 allow_colorbar=False, cmap='autumn')
imshow_node_grid(grid, overlay_landslide3, color_for_closed='None',
                 allow_colorbar=False, cmap='winter')
imshow_node_grid(grid, overlay_landslide4, color_for_closed='None',
                 allow_colorbar=False,cmap='summer')
cs = pylab.contour(elev, extent=[0,42500, 0,32500], hold='on', colors='black')
pylab.clabel(cs, inline=True, fmt='%1i', fontsize=10, manual=manual_locals)
imshow_node_grid(grid, overlay_watershed, color_for_closed='None',
                 allow_colorbar=False, cmap='spring')

This map shows slope with mapped landslides (blue - debris avalanches, cyan - falls/topples,
red - debris torrents, and gren - slumps/creeps).

Plot soil thickness with contours

In [ ]:
plt.figure('Soil Thickness')
imshow_node_grid(grid, 'soil__thickness', cmap='copper_r',
                 grid_units=('coordinates', 'coordinates'), shrink=0.75,
                 var_name='Soil Thickness', var_units='m')
cs = pylab.contour(elev, extent=[0,42500, 0,32500], hold='on', colors='black')
pylab.clabel(cs, inline=True, fmt='%1i', fontsize=10, manual=manual_locals)
imshow_node_grid(grid, overlay_watershed, color_for_closed='None',
                 allow_colorbar=False, cmap='spring')

Plot cohesion mode with contours

In [ ]:
plt.figure('Cohesion')
imshow_node_grid(grid, 'soil__mode_total_cohesion', cmap='Greens',
                 grid_units=('coordinates', 'coordinates'), shrink=0.75,
                 var_name='Cohesion', var_units='Pa')
cs = pylab.contour(elev, extent=[0,42500, 0,32500], hold='on', colors='black')
pylab.clabel(cs, inline=True, fmt='%1i', fontsize=10, manual=manual_locals)
imshow_node_grid(grid, overlay_watershed, color_for_closed='None',
                 allow_colorbar=False, cmap='spring')

This map shows total cohesion, where the dark green areas represent forests.

Plot transmissivity with contours

In [ ]:
plt.figure('Transmissivity')
imshow_node_grid(grid, 'soil__transmissivity', cmap='Purples',
                 grid_units=('coordinates', 'coordinates'), shrink=0.75,
                 var_name='Transmissivity', var_units='m2/d')
cs = pylab.contour(elev, extent=[0,42500, 0,32500], hold='on', colors='black')
pylab.clabel(cs, inline=True, fmt='%1i', fontsize=10, manual=manual_locals)
imshow_node_grid(grid, overlay_watershed, color_for_closed='None',
                 allow_colorbar=False, cmap='spring')

This map shows soil transmissivity where the darker colors indicate higher transmissivity in the ticker soils of the valley bottoms.

Plot relative wetness with contours

In [ ]:
plt.figure('Mean Relative Wetness')
imshow_node_grid(grid, 'soil__mean_relative_wetness', cmap='YlGnBu',
                 grid_units=('coordinates', 'coordinates'),
                 shrink=0.75, var_name='Relative Wetness',
                 var_units='no units')
cs = pylab.contour(elev, extent=[0,42500, 0,32500], hold='on', colors='gray')
pylab.clabel(cs, inline=True, fmt='%1i', fontsize=10, manual=manual_locals)
imshow_node_grid(grid, overlay_watershed, color_for_closed='None',
                 allow_colorbar=False, cmap='spring')

This map shows the relative wetness as high throughout the areas because we modeled the annual peak recharge, which is esssentially the worst case conditions that might lead to instability.

Plot mean factor-of-safety with contours. Note that mean factor-of-safety may not be a good indication of stability if the factor-of-safety is highly right-skewed.  

In [ ]:
plt.figure('Mean Factor of Safety')
imshow_node_grid(grid, 'landslide__mean_factor_of_safety', cmap='OrRd_r',
                 grid_units=('coordinates', 'coordinates'), vmax=5.,
                 shrink=0.75, var_name='Factor of Safety',
                 var_units='no units')
cs = pylab.contour(elev, extent=[0,42500, 0,32500], hold='on', colors='black')
pylab.clabel(cs, inline=True, fmt='%1i', fontsize=10, manual=manual_locals)
imshow_node_grid(grid, overlay_watershed, color_for_closed='None',
                 allow_colorbar=False, cmap='spring')

This map shows the mean factor-of-safety at each node based on 25 (number of simulations) calculations of factor-of-safety.

Plot probability of failure with contours

In [ ]:
plt.figure('Probability of Failure')
imshow_node_grid(grid, 'landslide__probability_of_failure', cmap='OrRd',
                 grid_units=('coordinates', 'coordinates'), shrink=0.75,
                 var_name='Probability of Failure', var_units='no units')
cs = pylab.contour(elev, extent=[0,42500, 0,32500], hold='on', colors='black')
pylab.clabel(cs, inline=True, fmt='%1i', fontsize=10, manual=manual_locals)
imshow_node_grid(grid, overlay_watershed, color_for_closed='None',
                 allow_colorbar=False, cmap='spring')

This map shows the probability of failure based on then number of simulations where the factor-of-safety was <= 1.0 out all the simulations. The probability tends to be higer in the upper portions of the watershed where cohesion is less and soils are shallower, quickly saturating.

Plot the factor-of-safety histogram at one node and check its probability of failure.

In [ ]:
example_FS_dist = LS_prob.landslide__factor_of_safety_histogram[
    core_nodes[46]]  # node array index, not node id
plt.figure('Ex_FS_Distribution')
plt.hist(example_FS_dist, 22)
plt.plot([1, 1], [0, 4], color='r', linestyle='-', linewidth=3)
plt.title('Sample FS Distribution at one node')
plt.ylabel('Frequency')
plt.xlabel('Calculated Factor-of-Safety')

grid['node']['landslide__probability_of_failure'][core_nodes[46]]

This shows a histogram of the calculated factor-of-safety values at one node.  The probability of failure is determined as the number of calculated factor-of-safety values <= 1.0 (red line) divided by the total number of simulated factor-of-safety values (i.e., number of simulations user specified). The probability value will vary based on the randomness in generated data fields above and input parameter distributions generated by the component from which the simulations are sampling when calculating each factor-of-safety value.

To save plots, run "plt.savefig('figure_name.png', dpi=300)" and put in your own path and figure name.

Now it is your turn to explore the component and your particular site.  For example, try reducing your cohesion by 50% (perhaps a low intensity fire) or increasing your recharge by 20% (wetter conditions). Then rerun the component to see how the probabilty of failure changes.